---
description: This module provides functions that convert FEEMSResult to proto buffer
  message
output-file: convertfeemsresulttoproto.html
title: Converter FEEMSResult to proto

---



In [1]:
import os
import random

from MachSysS.convert_feems_result_to_proto import FEEMSResultConverter
from MachSysS.convert_to_feems import convert_proto_propulsion_system_to_feems
from MachSysS.gymir_result_pb2 import GymirResult, SimulationInstance
from MachSysS.system_structure_pb2 import MachinerySystem
from RunFeemsSim.machinery_calculation import MachineryCalculation

In [2]:


# Create a random GymirResult and load the system
def create_gymir_result() -> GymirResult:
    return GymirResult(
        name="test",
        auxiliary_load_kw=500 * random.random(),
        result=[
            SimulationInstance(epoch_s=100 * index + 1, power_kw=1000 * random.random())
            for index, _ in enumerate(range(10))
        ],
    )


path = os.path.join("tests", "system_proto.mss")
with open(path, "rb") as file:
    system_proto = MachinerySystem()
    system_proto.ParseFromString(file.read())

In [3]:
# Test for the case of no ICE in the system
import os

import MachSysS.system_structure_pb2 as sys_structure_pb2
from MachSysS.convert_feems_result_to_proto import FEEMSResultConverter
from MachSysS.convert_to_feems import convert_proto_propulsion_system_to_feems
from MachSysS.gymir_result_pb2 import TimeSeriesResult
from MachSysS.utility import retrieve_machinery_system_from_file
from RunFeemsSim.machinery_calculation import MachineryCalculation

# Import the system from the file
path = os.path.join("tests", "electric_propulsion_system.mss")
system_proto = retrieve_machinery_system_from_file(path)
system_proto_copy = sys_structure_pb2.MachinerySystem()
system_proto_copy.CopyFrom(system_proto)
# Remove genset from the system
for switchboard_idx, switchboard in enumerate(system_proto.electric_system.switchboards):
    for subsystem in switchboard.subsystems:
        if subsystem.component_type == sys_structure_pb2.Subsystem.ComponentType.GENSET:
            print(f"removing genset: {subsystem.name}")
            system_proto_copy.electric_system.switchboards[switchboard_idx].subsystems.remove(
                subsystem
            )
system_feems = convert_proto_propulsion_system_to_feems(system_proto_copy)
# Import the time series data
path = os.path.join("tests", "time_series_result.pb")
time_series_pb = TimeSeriesResult()
with open(path, "rb") as file:
    time_series_pb.ParseFromString(file.read())

# Create a machinery calculation object
machinery_calculation = MachineryCalculation(
    feems_system=system_feems, maximum_allowed_power_source_load_percentage=80
)
res = machinery_calculation.calculate_machinery_system_output_from_time_series_result(
    time_series=time_series_pb,
)
res_converter = FEEMSResultConverter(
    feems_result=res, system_feems=system_feems, time_series_input=time_series_pb
)
res_proto = res_converter.get_feems_result_proto()
print(res_proto)

removing genset: Genset 1
removing genset: Genset 2
removing genset: Genset 3
removing genset: Genset 4
electric_system {
  duration_s: 10799940
  multi_fuel_consumption_total_kg {
    fuels {
      fuel_type: HYDROGEN
      fuel_origin: RENEWABLE_NON_BIO
      fuel_specified_by: IMO
      mass_or_mass_fraction: 4082989.3255475312
      lhv_mj_per_g: 0.12
    }
  }
  running_hours_fuel_cell_total_hr: 5268.75
  co2_emission_total_kg {
  }
  detailed_result {
    component_name: "Fuel cell system 1"
    multi_fuel_consumption_kg {
      fuels {
        fuel_type: HYDROGEN
        fuel_origin: RENEWABLE_NON_BIO
        fuel_specified_by: IMO
        mass_or_mass_fraction: 2029071.5767343191
        lhv_mj_per_g: 0.12
      }
    }
    running_hours_h: 2268.7666666666669
    co2_emissions_kg {
    }
    component_type: "FUEL_CELL_SYSTEM"
    rated_capacity: 3150
    rated_capacity_unit: "kW"
    switchboard_id: 1
    fuel_consumer_type: "FUEL_CELL"
  }
  detailed_result {
    component_nam